# Logistic Regression

## Data preprocessing

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold

In [5]:
game_data = pd.read_pickle("../flatten_games.pickle")
game_data_keys = game_data.keys()
print(len(game_data_keys))

12325


In [16]:
print(game_data[list(game_data.keys())[0]].keys())
game_data[list(game_data.keys())[0]]

dict_keys(['game_page_url', 'have_att', 'start_time', 'day', 'month', 'is_federal_holiday', 'duration', 'venue', 'at_night', 'on_grass', 'weather_description', 'temperature', 'wind_speed', 'weather', 'precipitation', 'umpires', 'attendance', 'team1_name', 'team1_pre_win', 'team1_pre_loss', 'team1_pre_win_pct', 'players', 'team1_streak', 'team2_name', 'team2_pre_win', 'team2_pre_loss', 'team2_pre_win_pct', 'team2_streak'])


{'game_page_url': 'https://www.baseball-reference.com/boxes/CHN/CHN201504050.shtml',
 'have_att': True,
 'start_time': datetime.datetime(2015, 4, 5, 19, 17),
 'day': 'Sunday',
 'month': 'April',
 'is_federal_holiday': 0,
 'duration': datetime.timedelta(seconds=11040),
 'venue': 'Wrigley Field',
 'at_night': True,
 'on_grass': True,
 'weather_description': ' 44° F, Wind 7mph in from Centerfield, Sunny.',
 'temperature': 44,
 'wind_speed': 7,
 'weather': 'in Dome',
 'precipitation': 3,
 'umpires': ' HP - Mike Winters, 1B - Mark Wegner, 2B - Marty Foster, 3B - Mike Muchlinski.',
 'attendance': 35055,
 'team1_name': 'StLouisCardinals',
 'team1_pre_win': 0,
 'team1_pre_loss': 0,
 'team1_pre_win_pct': 0,
 'players':                          Batting AB  R  H RBI BB SO PA    BA   OBP  ...  \
 0               Dexter Fowler CF  4  0  1   0  0  2  4  .250  .250  ...   
 1                 Jorge Soler RF  4  0  0   0  0  1  4  .000  .000  ...   
 2               Anthony Rizzo 1B  3  0  0   0  1  2 

In [19]:
df_game_data = pd.DataFrame()
for game_key in game_data_keys:
    # game_key = list(game_data.keys())[0]
    game = game_data[game_key]
    df_game = pd.DataFrame([game])
    # print(df_game.head())
    df_game_data = pd.concat([df_game_data, df_game])
df_game_data.head()

,game_page_url,have_att,start_time,day,month,is_federal_holiday,duration,venue,at_night,on_grass,...,team1_pre_win,team1_pre_loss,team1_pre_win_pct,players,team1_streak,team2_name,team2_pre_win,team2_pre_loss,team2_pre_win_pct,team2_streak
0,https://www.baseball-reference.com/boxes/CHN/C...,True,2015-04-05 19:17:00,Sunday,April,0,0 days 03:04:00,Wrigley Field,True,True,...,0,0,0.0,Batting AB R H RBI ...,0,ChicagoCubs,0,0,0.0,0
0,https://www.baseball-reference.com/boxes/ARI/A...,True,2015-04-06 19:13:00,Monday,April,0,0 days 03:18:00,Chase Field,True,True,...,0,0,0.0,Batting AB R H RBI BB ...,0,ArizonaDiamondbacks,0,0,0.0,0
0,https://www.baseball-reference.com/boxes/CIN/C...,True,2015-04-06 16:10:00,Monday,April,0,0 days 02:30:00,Great American Ball Park,False,True,...,0,0,0.0,Batting AB R H RBI BB SO P...,0,CincinnatiReds,0,0,0.0,0
0,https://www.baseball-reference.com/boxes/DET/D...,True,2015-04-06 13:08:00,Monday,April,0,0 days 02:30:00,Comerica Park,False,True,...,0,0,0.0,Batting AB R H RBI BB SO PA ...,0,DetroitTigers,0,0,0.0,0
0,https://www.baseball-reference.com/boxes/HOU/H...,True,2015-04-06 18:10:00,Monday,April,0,0 days 02:30:00,Minute Maid Park,True,True,...,0,0,0.0,Batting AB R H RBI BB SO PA ...,0,HoustonAstros,0,0,0.0,0


In [26]:
df_game_data = df_game_data.reset_index(drop=True)
df_game_data = df_game_data[['game_page_url', 'have_att', 'start_time', 'day', 'month', 
'is_federal_holiday', 'duration', 'venue', 'at_night', 'on_grass', 
'weather_description', 'temperature', 'wind_speed', 'weather', 'precipitation', 
'umpires', 'attendance', 'team1_name', 'team1_pre_win', 'team1_pre_loss', 'team1_pre_win_pct', 'players', 'team1_streak', 
'team2_name', 'team2_pre_win', 'team2_pre_loss', 'team2_pre_win_pct', 'team2_streak']]
df_game_data.head()

,game_page_url,have_att,start_time,day,month,is_federal_holiday,duration,venue,at_night,on_grass,...,team1_pre_win,team1_pre_loss,team1_pre_win_pct,players,team1_streak,team2_name,team2_pre_win,team2_pre_loss,team2_pre_win_pct,team2_streak
0,https://www.baseball-reference.com/boxes/CHN/C...,True,2015-04-05 19:17:00,Sunday,April,0,0 days 03:04:00,Wrigley Field,True,True,...,0,0,0.0,Batting AB R H RBI ...,0,ChicagoCubs,0,0,0.0,0
1,https://www.baseball-reference.com/boxes/ARI/A...,True,2015-04-06 19:13:00,Monday,April,0,0 days 03:18:00,Chase Field,True,True,...,0,0,0.0,Batting AB R H RBI BB ...,0,ArizonaDiamondbacks,0,0,0.0,0
2,https://www.baseball-reference.com/boxes/CIN/C...,True,2015-04-06 16:10:00,Monday,April,0,0 days 02:30:00,Great American Ball Park,False,True,...,0,0,0.0,Batting AB R H RBI BB SO P...,0,CincinnatiReds,0,0,0.0,0
3,https://www.baseball-reference.com/boxes/DET/D...,True,2015-04-06 13:08:00,Monday,April,0,0 days 02:30:00,Comerica Park,False,True,...,0,0,0.0,Batting AB R H RBI BB SO PA ...,0,DetroitTigers,0,0,0.0,0
4,https://www.baseball-reference.com/boxes/HOU/H...,True,2015-04-06 18:10:00,Monday,April,0,0 days 02:30:00,Minute Maid Park,True,True,...,0,0,0.0,Batting AB R H RBI BB SO PA ...,0,HoustonAstros,0,0,0.0,0


In [30]:
df_game_data['players'][0]

,Batting,AB,R,H,RBI,BB,SO,PA,BA,OBP,...,WPA,aLI,WPA+,WPA-,cWPA,acLI,RE24,PO,A,Details
0,Dexter Fowler CF,4,0,1,0,0,2,4,.250,.250,...,-0.033,1.24,0.065,-0.098%,-0.02%,1.35,-0.3,2,0,2B
1,Jorge Soler RF,4,0,0,0,0,1,4,.000,.000,...,-0.094,1.11,0.000,-0.094%,-0.06%,1.21,-0.9,0,0,
2,Anthony Rizzo 1B,3,0,0,0,1,2,4,.000,.250,...,-0.101,0.93,0.012,-0.113%,-0.07%,1.01,-0.9,7,1,
3,Starlin Castro SS,4,0,1,0,0,1,4,.250,.250,...,-0.071,0.89,0.014,-0.084%,-0.05%,0.97,-0.7,4,0,
4,Chris Coghlan LF,4,0,1,0,0,1,4,.250,.250,...,0.005,0.84,0.067,-0.062%,0.00%,0.92,0.0,1,0,2B
5,Mike Olt 3B,4,0,0,0,0,2,4,.000,.000,...,-0.094,0.84,0.000,-0.094%,-0.06%,0.92,-0.9,0,1,
6,David Ross C,2,0,1,0,0,0,2,.500,.500,...,0.032,1.17,0.065,-0.034%,0.02%,1.27,0.3,8,1,2B
7,Jason Motte P,0,0,0,0,0,0,0,,,...,,,,,,,,0,0,
8,Neil Ramirez P,0,0,0,0,0,0,0,,,...,,,,,,,,0,0,
9,Arismendy Alcantara PH-2B,1,0,0,0,1,1,2,.000,.500,...,0.039,0.71,0.043,-0.004%,0.03%,0.77,0.3,0,1,


In [7]:
target_player = pd.read_csv("../data/players/players_all_star_salary_5000000.csv")["Name"].to_numpy()
target_player[0:5]

array(['Adam Jones', 'Chris Sale', 'Aroldis Chapman', 'Miguel Cabrera',
       'David Price'], dtype=object)